In [1]:
import yfinance as yf
import pandas as pd
from ipywidgets import Output
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import precision_score, accuracy_score
predictors = []

In [2]:
stocks = ["^gspc",'avgo']
data =pd.DataFrame([])
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
first = True
               
for stock in stocks:
    stk = yf.Ticker(stock)  
    stock_data = stk.history(period='max') 
    stock_data=stock_data.loc["1990-1-1":].copy()
    del stock_data["Dividends"]
    del stock_data["Stock Splits"] 
    del stock_data["Open"]
    del stock_data["High"]
    del stock_data["Low"]
    stock_data["Tomarrow"] = stock_data["Close"].shift(-1)
    stock_data["Target"] = (stock_data["Close"]<stock_data["Tomarrow"]).astype(int)
    
    horizons = [2,5,60,250,1000]
    # predictors = []
    
    if  first:
        first = False
    else:
        for horizon in horizons:
            rolling_averages = stock_data.rolling(horizon).mean()

            ratio_column = f"{stock}_Close_Avg_{horizon}"
            stock_data[ratio_column] = stock_data["Close"] / rolling_averages["Close"]

            trend_column = f"{stock}_Trend_{horizon}"
            stock_data[trend_column] = stock_data.shift(1).rolling(horizon).sum()["Target"]

    #         v_ratio_col =f"{stock}_Vol_Avg_{horizon}"
    #         stock_data[v_ratio_col] = stock_data["Volume"]/rolling_averages["Volume"]

            predictors+= [ratio_column, trend_column] 

            # print("delete pred name")
            # del stock_data['Close']
            # del stock_data['Volume']
            # del stock_data['Target']
        data = pd.concat([data,stock_data],ignore_index = False,axis=1)


In [3]:
data.tail(4)

,Close,Volume,Tomarrow,Target,avgo_Close_Avg_2,avgo_Trend_2,avgo_Close_Avg_5,avgo_Trend_5,avgo_Close_Avg_60,avgo_Trend_60,avgo_Close_Avg_250,avgo_Trend_250,avgo_Close_Avg_1000,avgo_Trend_1000
Date,,,,,,,,,,,,,,
2024-03-05 00:00:00-05:00,1342.75,3604000,1350.00,1,0.98,1.00,1.00,3.00,1.14,36.00,1.51,135.00,2.43,539.00
2024-03-06 00:00:00-05:00,1350.00,4319600,1407.01,1,1.00,1.00,0.99,4.00,1.14,36.00,1.52,135.00,2.43,540.00
2024-03-07 00:00:00-05:00,1407.01,6283200,1308.72,0,1.02,2.00,1.02,4.00,1.18,36.00,1.58,136.00,2.53,540.00
2024-03-08 00:00:00-05:00,1308.72,7390200,NaN,0,0.96,1.00,0.96,3.00,1.09,35.00,1.46,136.00,2.35,540.00


In [4]:
data=data.dropna()

In [5]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

train = data.iloc[:-100]
test = data.iloc[-100:]

In [6]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [7]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [8]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [9]:
predictions = backtest(data, model, predictors)

In [10]:
predictions["Predictions"].value_counts()

Predictions
0.00    141
1.00     30
Name: count, dtype: int64

In [11]:
prec = precision_score(predictions["Target"], predictions["Predictions"])
acc = accuracy_score(predictions["Target"], predictions["Predictions"])

In [12]:
print('precision: ',prec)
print('accuracy: ',acc)

precision:  0.5666666666666667
accuracy:  0.47368421052631576


In [13]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
1   0.55
0   0.45
Name: count, dtype: float64

In [14]:
pd.concat([predictions.tail(15),data.tail(15)],axis=1)

,Target,Predictions,Close,Volume,Tomarrow,Target,avgo_Close_Avg_2,avgo_Trend_2,avgo_Close_Avg_5,avgo_Trend_5,avgo_Close_Avg_60,avgo_Trend_60,avgo_Close_Avg_250,avgo_Trend_250,avgo_Close_Avg_1000,avgo_Trend_1000
Date,,,,,,,,,,,,,,,,
2024-02-15 00:00:00-05:00,0,0.00,1265.07,2187300,1245.48,0,1.00,2.00,1.00,3.00,1.14,33.00,1.48,135.00,2.34,537.00
2024-02-16 00:00:00-05:00,0,0.00,1245.48,2126300,1226.55,0,0.99,1.00,0.99,2.00,1.12,32.00,1.46,135.00,2.30,537.00
2024-02-20 00:00:00-05:00,1,1.00,1226.55,2632900,1227.45,1,0.99,0.00,0.98,2.00,1.10,32.00,1.43,135.00,2.26,537.00
2024-02-21 00:00:00-05:00,1,0.00,1227.45,2064000,1304.90,1,1.00,1.00,0.99,3.00,1.10,33.00,1.43,136.00,2.26,537.00
2024-02-22 00:00:00-05:00,0,0.00,1304.90,4081900,1296.37,0,1.03,2.00,1.04,3.00,1.16,33.00,1.51,136.00,2.40,538.00
2024-02-23 00:00:00-05:00,1,0.00,1296.37,2311500,1309.13,1,1.00,1.00,1.03,2.00,1.15,33.00,1.50,136.00,2.38,537.00
2024-02-26 00:00:00-05:00,0,0.00,1309.13,1628200,1296.23,0,1.00,1.00,1.03,3.00,1.15,34.00,1.51,136.00,2.39,538.00
2024-02-27 00:00:00-05:00,0,0.00,1296.23,1212600,1289.42,0,1.00,1.00,1.01,3.00,1.13,34.00,1.49,135.00,2.37,538.00
2024-02-28 00:00:00-05:00,1,0.00,1289.42,1096700,1300.49,1,1.00,0.00,0.99,2.00,1.12,34.00,1.47,135.00,2.35,538.00


In [15]:
feature_imp=pd.Series( model.feature_importances_).sort_index(ascending=True)

In [16]:
pred=pd.Series(predictors)

In [17]:
print(pd.concat([pred,feature_imp],axis=1))

                     0    1
0     avgo_Close_Avg_2 0.16
1         avgo_Trend_2 0.02
2     avgo_Close_Avg_5 0.14
3         avgo_Trend_5 0.03
4    avgo_Close_Avg_60 0.15
5        avgo_Trend_60 0.06
6   avgo_Close_Avg_250 0.14
7       avgo_Trend_250 0.08
8  avgo_Close_Avg_1000 0.14
9      avgo_Trend_1000 0.09
